In [ ]:
import glob 
import os,re
import yt_dlp
import json
import time
import httplib2
import openai
import requests
import pinecone 
import pandas as pd
import numpy as np
import urllib.request
from langchain import OpenAI
from bs4 import BeautifulSoup
from bs4.element import Comment
from langchain.llms import OpenAIChat
from langchain.chains import VectorDBQA
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from bs4 import BeautifulSoup, SoupStrainer
from langchain.vectorstores import Pinecone
from youtubesearchpython import ChannelsSearch
from langchain.chains import VectorDBQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

`Get video metadata -` 
 
* https://pypi.org/project/youtube-search-python/

In [ ]:
# get channel 
channelsSearch = ChannelsSearch('lexfridman', limit = 10, region = 'US')
r = channelsSearch.result()
id = r['result'][0]['id']

# get all videos
channel_id = "UCSHZKyawb77ixDdsGog4iWA"
playlist = Playlist(playlist_from_channel_id(channel_id))
print(f'Videos Retrieved: {len(playlist.videos)}')

In [ ]:
# Get all episode data
stor_metadata=pd.DataFrame()
for v in playlist.videos:
    try:
        ep_number = int(v['title'].split("|")[-1].split("#")[-1])
        stor_metadata.loc[v['title'],'number']=ep_number
        stor_metadata.loc[v['title'],'link']=v['link']
        stor_metadata.loc[v['title'],'title']=v['title']
    except:
        print("Failed on %s", v['title'])

# Filter for those you want to dump audio
# Using existing @karpathy transcriptions for 1-325
# https://karpathy.ai/lexicap/0058-large.html
new_ep = stor_metadata[stor_metadata.number > 325]

`Download audio -` 

* https://github.com/yt-dlp/yt-dlp/wiki/Installation
* https://github.com/yt-dlp/yt-dlp

In [ ]:
# Save img and audio
for ix in new_ep.index:
    # get data 
    ep_number=int(new_ep.loc[ix,'number'])
    print("EPISODE: %s"%ep_number)
    img_url=new_ep.loc[ix,'img']
    ep_link=new_ep.loc[ix,'link']
    # write img 
    with open("img/%s.jpg"%str(ep_number), 'wb') as f:
        response = requests.get(img_url)
        f.write(response.content)
    # write audio
    ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'outtmpl': 'audio/%s.m4a'%str(ep_number),
    'noplaylist': True,
    'postprocessors': [{  
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }]}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download(ep_link)

# Save 
new_ep.reset_index().to_csv("audio_transcription/episodes.csv")

`Transcribe newer episodes`
 
* Do this on GPU, ideally 
* ~10-20 min / video on 2080Ti with `medium` model size

In [ ]:
! python run_whisper.py

`Scrape existing transcriptions`

* https://karpathy.ai/lexicap/index.html
* https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/qa_with_sources.html

In [ ]:
# Get all pages 
http = httplib2.Http()
status, response = http.request("https://karpathy.ai/lexicap/")
links = []
for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        links.append(link['href'])
links_tx = ["https://karpathy.ai/lexicap/"+l for l in links if "0" in l]

In [ ]:
# Get text -
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def get_text_and_title(url):
    html = urllib.request.urlopen(url).read()
    t=(text_from_html(html))
    title=t.split("|")[0].split("back to index")[1].strip()
    return t, title

# Get links -
def get_links(URL):
    http = httplib2.Http()
    status, response = http.request(URL)
    links = []
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            links.append(link['href'])
    links_clean = [l for l in links if "https" in l]
    return links_clean

# Get image -
def get_img(URL,title,episode_id):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    img_tags = soup.find_all('img')
    urls = [img['src'] for img in img_tags]
    for url in urls:
        response = requests.get(url)
        imgpath="../public/0%s.jpg"%episode_id
        with open(imgpath, 'wb') as f:
            if 'http' not in url:
                url = '{}{}'.format(site, url)
            response = requests.get(url)
            f.write(response.content)
    return imgpath

# Full pipeline - 
def pre_process(URL,episode_id):

    t,title=get_text_and_title(URL)
    links=get_links(URL)
    img=get_img(URL,title,episode_id)
    stor_chunk = pd.DataFrame()
    stor_chunk['chunks']= t.split("link |")
    stor_chunk['clean_chunks']=stor_chunk['chunks'].apply(lambda x: re.sub(r"[^a-zA-Z ]+", '', x)).apply(lambda x: x.strip())
    stor_chunk['links']=links
    all_text = stor_chunk['clean_chunks'].str.cat(sep=' ')
    return all_text, links, title

# Make splits - 
def make_splits(chunks,URL):

    # ID
    episode_id=URL.split("/")[-1].split("-")[0]

    # Pre-processing
    texts,links,title=pre_process(URL,episode_id)
    
    # Splits 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunks, 
                                                   chunk_overlap=50) 
    texts_recusive = text_splitter.split_text(texts)
    print(len(texts_recusive)) 

    # Metadata 
    N = len(texts_recusive) 
    bins = np.linspace(0, len(links)-1, N, dtype=int)
    sampled_links = [links[i] for i in bins]
    # Here we can add "link", "title", etc that can be fetched in the app 
    metadatas=[{"source":title + " " +link,"id":episode_id,"link":link,"title":title} for link in sampled_links]
    print(len(metadatas))
    return texts_recusive,metadatas,title,episode_id

In [ ]:
# *** Chunk size: key parameter *** 
chunks = 1500
# *** Chunk size: key parameter *** 
splits_all = [ ]
metadatas_all = [ ]
 
# Iterate 
stor=pd.DataFrame()
for page in links_tx:
    try:
        print("Writing: %s"%page)
        splits,metadatas,title,episode_id=make_splits(chunks,page)
        stor.loc[episode_id,'title']=title 
        with open('docs/%s.txt'%episode_id, "w") as f:
            for string in splits:
                f.write(string + "\n") 
        f.close()
        with open('metadatas/%s.json'%episode_id, "w") as f:
            json.dump(metadatas, f)
        f.close()
        splits_all.append(splits)
        metadatas_all.append(metadatas)
    except:
        print("Error on page: %s"%page)
    
# Combine all 
splits_combined = []
for sublist in splits_all:
    splits_combined.extend(sublist)
metadatas_combined = []
for sublist in metadatas_all:
    metadatas_combined.extend(sublist)

`Get newer transcripts`

In [ ]:
# Read the csv file
new_ep=pd.read_csv("audio_transcription/episodes.csv",index_col=None)

In [ ]:
# *** Chunk size: key parameter *** 
chunks = 1500
# *** Chunk size: key parameter *** 
splits_all_new = [ ]
metadatas_all_new = [ ]

for ix in new_ep.index:

    # Get data
    title=new_ep.loc[ix,'title']
    ep_number=int(new_ep.loc[ix,'number'])
    episode_id="0"+str(ep_number) # consistency w/ convention used in Karpathy transcription
    out_file_path='audio_transcription/%s.txt'%str(episode_id)
    transcript=pd.read_csv(out_file_path,sep='\t',header=None)
    transcript.columns=['links','time','chunks']
    transcript['clean_chunks']=transcript['chunks'].astype(str).apply(lambda x: x.strip())
    links = list(transcript['links'])
    texts = transcript['clean_chunks'].str.cat(sep=' ')

    # Splits 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunks, 
                                                   chunk_overlap=50) 
    splits = text_splitter.split_text(texts)
    print(len(splits)) 

    # Metadata 
    N = len(splits) 
    bins = np.linspace(0, len(links)-1, N, dtype=int)
    sampled_links = [links[i] for i in bins]
    
    # Here we can add "link", "title", etc that can be fetched in the app 
    metadatas=[{"source":title + " " +link,"id":episode_id,"link":link,"title":title} for link in sampled_links]
    print(len(metadatas)) 

    splits_all_new.append(splits)
    metadatas_all_new.append(metadatas)

# Combine all 
splits_combined_new = []
for sublist in splits_all_new:
    splits_combined_new.extend(sublist)
metadatas_combined_new = []
for sublist in metadatas_all_new:
    metadatas_combined_new.extend(sublist)

`Read in the saved files`

In [ ]:
def get_saved_doc(doc):

    dir_path, file_name = os.path.split(doc)
    name, ext = os.path.splitext(file_name)

    with open(doc, "r") as f:
        my_strings = f.readlines()
    f.close()

    doc_to_read_txt = [string.strip() for string in my_strings]

    with open('metadatas/%s.json'%name, "r") as f:
        doc_to_read_metadatas = json.load(f)
    f.close()

    return doc_to_read_txt,doc_to_read_metadatas

# Read files in 
docs = glob.glob("docs/*")
splits_all_read_fromdisk = []
metadatas_all_readfromdisk = []
for doc_to_read in docs:
    doc_to_read_txt,doc_to_read_metadatas = get_saved_doc(doc_to_read)
    splits_all_read_fromdisk.append(doc_to_read_txt)
    metadatas_all_readfromdisk.append(doc_to_read_metadatas)

# Combine 
splits_combined_fromdisk = []
metadatas_combined_fromdisk = []
for sublist in splits_all_read_fromdisk:
    splits_combined_fromdisk.extend(sublist)
for sublist in metadatas_all_readfromdisk:
    metadatas_combined_fromdisk.extend(sublist)

# Smaller index for testing
oriol_text,oriol_metadatas=get_saved_doc('docs/0306.txt')
demis_text,demis_metadatas=get_saved_doc('docs/0299.txt')
txt_test=oriol_text+demis_text
metadatas_test=oriol_metadatas+demis_metadatas
print(len(oriol_text))
print(len(oriol_metadatas))
print(len(txt_test))
print(len(metadatas_test))

`Embedding: small-scale test`

PY support - 

https://langchain.readthedocs.io/en/latest/reference/modules/vectorstore.html

JS support - 

https://hwchase17.github.io/langchainjs/docs/modules/indexes/vector_stores/supabase/

In [ ]:
# Test for Chroma 
embeddings = OpenAIEmbeddings()
c_test = Chroma.from_texts(txt_test, embeddings, metadatas=metadatas_test)

In [ ]:
# Test for FAISS
embeddings = OpenAIEmbeddings()
f_test = FAISS.from_texts(txt_test, embeddings, metadatas=metadatas_test)

In [ ]:
# Test for pinecone 
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),  
    environment="us-east1-gcp"  
)
index_name = "lex-test"
embeddings = OpenAIEmbeddings()
# Write new DB
# p_test = Pinecone.from_texts(txt_test, embeddings, index_name=index_name, metadatas=metadatas_test)
# Read existing DB
p_test = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

`Embed full dataset`

* Takes 1-2 hrs w/ some babysitting (due to OpenAI rate limits, etc)

https://docs.pinecone.io/docs/insert-data
https://langchain.readthedocs.io/en/latest/modules/indexes/how_to_guides.html
https://langchain.readthedocs.io/en/latest/reference/modules/vectorstore.html

In [ ]:
# Chroma
for i, dict in enumerate(metadatas_combined):
    try:
        json_str = json.dumps(dict)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError in object {i}: {e}")
metadatas_combined_json_str=[json.dumps(dict) for dict in metadatas_combined]
embeddings = OpenAIEmbeddings()
chroma_docstore = Chroma.from_texts(splits_combined, embeddings, metadatas=metadatas_combined_json_str,persist_directory="index/",collection_name="lex-gpt")


In [ ]:
# FAISS
faiss_docstore = FAISS.from_texts(splits_combined, embeddings, metadatas=metadatas_combined)
faiss_docstore.save_local('index/lex_faiss')

In [ ]:
# Pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),  
    environment="us-east1-gcp"  
)
index_name = "lex-gpt"
embeddings = OpenAIEmbeddings()
# p = Pinecone.from_texts(splits_combined[0:2], embeddings, index_name=index_name, metadatas=metadatas_combined[0:2])
p = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

In [ ]:
# Add all data 
import math, time 
chunk_size = 100
num_chunks = math.ceil(len(splits_combined) / chunk_size)
for i in range(last_chunk,num_chunks):
    
    print(i)
    start_time = time.time()
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, len(splits_combined))
    # Extract the current chunk
    current_splits = splits_combined[start_idx:end_idx]
    current_metadatas = metadatas_combined[start_idx:end_idx]
    # Add the current chunk to the vector database
    p.add_texts(texts = current_splits, metadatas=current_metadatas)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

In [ ]:
# Pinecone
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),  
    environment="us-east1-gcp"  
)
index_name = "lex-gpt-new"
embeddings = OpenAIEmbeddings()
# p = Pinecone.from_texts(splits_combined_new[0:2], embeddings, index_name=index_name, metadatas=metadatas_combined_new[0:2])
# p = Pinecone.from_existing_index(index_name=index_name,embedding=embeddings)

In [ ]:
# Add all data 
import math, time 
chunk_size = 100
last_chunk = 14
num_chunks = math.ceil(len(splits_combined_new) / chunk_size)
for i in range(last_chunk,num_chunks):
    
    print(i)
    start_time = time.time()
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, len(splits_combined_new))
    # Extract the current chunk
    current_splits = splits_combined_new[start_idx:end_idx]
    current_metadatas = metadatas_combined_new[start_idx:end_idx]
    # Add the current chunk to the vector database
    p.add_texts(texts = current_splits, metadatas=current_metadatas)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

`Run chain`

https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/qa_with_sources.html#
https://langchain.readthedocs.io/en/latest/modules/indexes/chain_examples/vector_db_qa_with_sources.html

In [ ]:
def run_qa_chain(llm,query,docstore):

    start_time = time.time()
    docs = docstore.similarity_search(query)
    chain = load_qa_chain(llm, chain_type="stuff")
    a = chain.run(input_documents=docs, question=query)
    print(a["output_text"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

def run_qa_sources_chain(llm,query,docstore):
    
    start_time = time.time()
    docs = docstore.similarity_search(query)
    chain = load_qa_with_sources_chain(llm, chain_type="stuff")
    a = chain({"input_documents": docs, "question": query})
    print(a["output_text"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

def run_vectordbqa_chain(llm,query,docstore):

    start_time = time.time()
    chain = VectorDBQA.from_chain_type(llm, chain_type="stuff", vectorstore=docstore)
    a = chain({"query": query})
    print(a['result'])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

def run_vectordb_sources_chain(llm,query,docstore):

    start_time = time.time()
    chain = VectorDBQAWithSourcesChain.from_chain_type(llm, chain_type="stuff", vectorstore=docstore)
    a = chain({"question": query},return_only_outputs=True)
    print(a["answer"])
    print(a["sources"])
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")
    print("--------")

llm = OpenAIChat(temperature=0)
q = "What is the future path for AGI?"
run_qa_chain(llm,q,p_oriol)
run_qa_sources_chain(llm,q,p_oriol) 
run_vectordbqa_chain(llm,q,p_oriol)
run_vectordb_sources_chain(llm,q,p_test)
run_vectordb_sources_chain(llm,q,p)
